In [1]:
source("~/smartas/pipeline/scripts/variablesAndFunctions.r")
library(gtools)

# entropy calculation only among those genes undergoing an isoform switch
# entropy measures the homogeneity in the distribution of the patients 
# in different cancer types
shannon.entropy <- function(p){
  if (invalid(p) || min(p) < 0 || sum(p) <= 0)
    return(NaN)
  p.norm <- p[p>0]/sum(p)
  H <- -sum(log2(p.norm)*p.norm)
  if (H == 0){
    0
  } else {
    maxH <- log2(length(p.norm))
    H/maxH
  }
}

get.entropy <- function(genes,ntxs,ttxs,patients,mask){
    m <- ifelse(is.null(mask),FALSE,mask)
    switches.agg <- data.frame(Gene=genes,nTx=ntxs,tTx=ttxs,Patients=patients) %>%
        filter(m) %>%
        group_by(Gene,nTx,tTx) %>%
        summarise(Patients=sum(Patients)) %>%
        mutate(p=Patients/sum(Patients))
    shannon.entropy(switches.agg$p)
}

get.uniq.switches <- function(genes,ntxs,ttxs,mask){
    m <- ifelse(is.null(mask),FALSE,mask)
    as.data.frame(cbind(genes,ntxs,ttxs)) %>%
        filter(m) %>%
        unique %>%
        nrow
}

get.uniq.patients <- function(genes,ntxs,ttxs,patients,mask){
    m <- ifelse(is.null(mask),FALSE,mask)
    switches <- data.frame(Gene=genes,nTx=ntxs,tTx=ttxs,Patients=patients) %>%
        filter(m)
    sum(switches$Patients)
}

my.binomial.test <- function(x,testNumber){ 
    if (!is.na(x[1]) & ! is.na(x[2]))
        binom.test(x[1],testNumber,x[2],"greater")$p.value
    else
        NA
}


Attaching package: ‘dplyr’

The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# Read switches
switches <- read_tsv("../data/pancancer/candidateList_info.tumorSplit.tsv")

# Read feature information
pfam.agg <- read_tsv("../data/mutations/proteome_features.txt") %>%
    filter(Analysis == "Pfam") %>%
    group_by(Feature) %>%
    summarize(ProteomeCounts=length(Feature),
              TotalLength = sum(FeatureLength)) %>%
    # get expected frequencies
    mutate(ExpectedSwitchFrequency = ProteomeCounts/sum(ProteomeCounts),
           ExpectedMutFrequency = TotalLength/sum(TotalLength))

## Calculate enrichment in switches

In [3]:
# Read structural changes
pfam.switch_info <- read_tsv('../data/structural_analysis/structural_features.onlyModels.tsv') %>%
    filter(Random=="NonRandom" & Analysis=="Pfam" & WhatsHappenning!="Nothing") %>%
    # Use switch information to characterize structural information
    merge(switches,by.x=c("Cancer","Gene","Symbol","nTx","tTx"),
          by.y=c("Tumor","GeneId","Symbol","Normal_transcript","Tumor_transcript")) %>%
    group_by(Feature) %>%
    summarize(LostNum = sum((WhatsHappenning=="Lost_in_tumor") * PatientNumber), 
              GainNum = sum((WhatsHappenning=="Gained_in_tumor") * PatientNumber), 
              H_g = get.entropy(Gene,nTx,tTx,PatientNumber,WhatsHappenning=="Gained_in_tumor"), 
              H_l = get.entropy(Gene,nTx,tTx,PatientNumber,WhatsHappenning=="Lost_in_tumor"), 
              patients_g = get.uniq.patients(Gene,nTx,tTx,PatientNumber,WhatsHappenning=="Gained_in_tumor"),
              switches_g = get.uniq.switches(Gene,nTx,tTx,WhatsHappenning=="Gained_in_tumor"),
              patients_l = get.uniq.patients(Gene,nTx,tTx,PatientNumber,WhatsHappenning=="Lost_in_tumor"),
              switches_l = get.uniq.switches(Gene,nTx,tTx,WhatsHappenning=="Lost_in_tumor"))

# aggregate switches
totalGains <- sum(pfam.switch_info$GainNum)
totalLosses <- sum(pfam.switch_info$LostNum)

# calculate statistics
pf <- merge(pfam.agg,pfam.switch_info,all=T) %>%
    mutate(fc_g = GainNum/totalGains/ExpectedSwitchFrequency,
           fc_l = LostNum/totalLosses/ExpectedSwitchFrequency) %>%
    mutate(., 
           p_g = apply(subset(.,select=c("GainNum","ExpectedSwitchFrequency")),1,my.binomial.test,totalGains),
           p_l = apply(subset(.,select=c("LostNum","ExpectedSwitchFrequency")),1,my.binomial.test,totalLosses)) %>%
    mutate(adjp_g = p.adjust(p_g),
           adjp_l = p.adjust(p_l)) %>%
    mutate(Name = gsub("|","@",Feature,fixed = T)) %>%
    mutate(Name = gsub("_"," ",Name,fixed = T)) %>%
    mutate(id = unlist(strsplit(Name,"@"))[c(T,F)], 
           Name = unlist(strsplit(Name,"@"))[c(F,T)])

Let's check the domains that are significantly enriched in switches and that affect more than one switch. The rationale behind the latter criteria is that if a domain is only affected by one switch, we have no-way to ensure it is domain specific rather than gene-specific.

In [4]:
sum(pf$adjp_l<0.05 & pf$switches_l>0,na.rm=T)
sum(pf$adjp_g<0.05 & pf$switches_g>0,na.rm=T)

[1] 590

[1] 108

## Description of the domains

In [5]:
# plot number of domains in each condition
df <- pf

df$Tag <- "Not altered"
df$Tag[df$adjp_g < 0.05 & pf$switches_g>1] <- "Gained"
df$Tag[df$adjp_l < 0.05 & pf$switches_l>1] <- "Lost"
df$Tag[df$adjp_g < 0.05  & pf$switches_g>1 & df$adjp_l < 0.05 & pf$switches_l>1] <- "Both"

table(df$Tag)


     Gained        Lost Not altered 
         68         335        4181 

### Comparison to mutations

In [6]:
# Read mutations
allMuts <- c("Frame_Shift_Del","Frame_Shift_Ins","In_Frame_Del","In_Frame_Ins","Missense_Mutation",
             "Nonsense_Mutation","Nonstop_Mutation","Frame_Shift_Del_out","Frame_Shift_Ins_out",
             "Nonsense_Mutation_out")
inFeatureMuts <- c("Frame_Shift_Del","Frame_Shift_Ins","In_Frame_Del","In_Frame_Ins",
                   "Missense_Mutation","Nonsense_Mutation","Nonstop_Mutation")

## calculate total mutations per domain
proteome.muts <- read_tsv("../data/mutations/proteome_mutations.txt") %>%
    filter(Analysis=="Pfam") %>%
    dcast(formula=Feature+Cancer+Transcript~Type, value.var="Patient") %>%
    group_by(Feature) %>%
    summarise(Frame_Shift_Del = sum(Frame_Shift_Del),
              Frame_Shift_Ins = sum(Frame_Shift_Ins),
              In_Frame_Del = sum(In_Frame_Del),
              In_Frame_Ins = sum(In_Frame_Ins),
              Missense_Mutation = sum(Missense_Mutation),
              Nonsense_Mutation = sum(Nonsense_Mutation),
              Nonstop_Mutation = sum(Nonstop_Mutation),
              Frame_Shift_Del_out = sum(Frame_Shift_Del_out),
              Frame_Shift_Ins_out = sum(Frame_Shift_Ins_out),
              Nonsense_Mutation_out = sum(Nonsense_Mutation_out)) %>%
    mutate(TotalMutations = rowSums(.[,inFeatureMuts]))

# enrichment test
pf.m <- merge(pfam.agg,proteome.muts) %>%
    mutate(fc_m = TotalMutations/sum(TotalMutations)/ExpectedMutFrequency,
           p_m = apply(.[,c("TotalMutations","ExpectedMutFrequency")],1, my.binomial.test, sum(TotalMutations)),
           adjp_m = p.adjust(p_m))

# create table with all the analysis
pf.all <- merge(pf,pf.m,all=T)

# save results
subset(pf.all,select=c("id","Feature","ProteomeCounts","ExpectedSwitchFrequency","LostNum","GainNum",
                   "fc_g","fc_l","fc_m","p_g","adjp_g","p_l","adjp_l","p_m","adjp_m","H_g","H_l",
                   "switches_g","switches_l","patients_g","patients_l")) %>%
    write_tsv("../data/structural_analysis/pfam_enrichment_analysis.tsv")

Aggregation function missing: defaulting to length


In [7]:
sum(pf.m$adjp_m < 0.05)

[1] 334

In [8]:


M <- pf.all$adjp_m < 0.05
S <- (pf.all$adjp_l < 0.05 & pf.all$switches_l > 1) | (pf.all$adjp_g < 0.05 & pf.all$switches_g > 1)

l <- rep("N", nrow(pf.all))
l[M] <- "M"
l[S] <- "S"
l[M&S] <- "MS"

m <- matrix(table(l),2,2)
fisher.test(m)
m %>%
    as.data.frame %>%
    set_colnames(c("M","NM")) %>%
    set_rownames(c("NS","S"))


	Fisher's Exact Test for Count Data

data:  m
p-value = 0.0008592
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.4004619 0.7920342
sample estimates:
odds ratio 
 0.5583655 


,M,NM
NS,287,3894
S,47,356


There is a higher coincidence than expected by chance. Hence, switches and mutations seem to affect the same protein domains.

In [9]:
# total number of domains observed
length(unique(c(pfam.agg$Feature,pf$Feature,pf.m$Feature)))

# number of domains for which switch has been observed
sum(pf$LostNum > 0 | pf$GainNum > 0,na.rm=T)

# number of domains for which a mutation has been observed
sum(pf.m$TotalMutations > 0)

[1] 4584

[1] 1776

[1] 4436

### GO term enrichment